In [ ]:
# Colab Notebook
# mount drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# For training an upsampling CNN
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

# Load the data from the .npy files
A = np.load("/content/drive/MyDrive/papagayo/upsampling/butcherbird_embeddings.npy").astype(np.float32)  # Shape (N, 8, 8)
B = np.load("/content/drive/MyDrive/papagayo/training_data/final_dataset_butcherbird.npy").astype(np.float32)  # Shape (N, 2049)

print(A.shape, B.shape)
# Convert to torch tensors
X = torch.tensor(A, dtype=torch.float32).unsqueeze(1)  # Adding channel dimension (N, 1, 8, 8)
y = torch.tensor(B, dtype=torch.float32)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create DataLoader for batching during training
train_data = TensorDataset(X_train, y_train)
val_data = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

# Check if GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define a CNN model
class PredictionCNN(nn.Module):
    def __init__(self):
        super(PredictionCNN, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)  # (1, 8, 8) -> (16, 8, 8)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1) # (16, 8, 8) -> (32, 8, 8)

        # Fully connected layers
        self.fc1 = nn.Linear(32 * 8 * 8, 256)  # 32 channels * 8 * 8 grid
        self.fc2 = nn.Linear(256, 2049)  # Output size is 2049

    def forward(self, x):
        # Apply convolutions with ReLU activations
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))

        # Flatten the output of convolution layers
        x = x.view(x.size(0), -1)  # Flatten (N, 32, 8, 8) -> (N, 32*8*8)

        # Fully connected layers
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)

        return x

# Instantiate the CNN model and move it to the device (GPU or CPU)
model = PredictionCNN().to(device)

# Define the loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 200
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)  # Move data to GPU

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, targets)

        # Backward pass
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print training progress
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(train_loader)}")

    # Validation phase
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)  # Move data to GPU
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()

        print(f"Validation Loss: {val_loss / len(val_loader)}")

# Save the trained model
torch.save(model.state_dict(), "/content/drive/MyDrive/papagayo/upsampling/prediction_cnn_model_butcherbirds.pth")